<a href="https://colab.research.google.com/github/chemicoPy/COVID-19_Data_visualization/blob/main/COVID19_Data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly
!pip install requests

In [2]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from pandas import DataFrame, Series
import math
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from datetime import datetime

In [3]:
raw = requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
data = pd.DataFrame(raw_json["features"])

data_list = data["attributes"].tolist()
dataset = pd.DataFrame(data_list)
dataset.set_index("OBJECTID")
dataset = dataset[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Last_Update"]]


def convertTime(t):
    t = int(t)
    return datetime.fromtimestamp(t)

dataset = dataset.dropna(subset=["Last_Update"])
dataset["Province_State"].fillna(value="", inplace=True)

dataset["Last_Update"]= dataset["Last_Update"]/1000
dataset["Last_Update"] = dataset["Last_Update"].apply(convertTime)

In [4]:
dataset

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Last_Update
0,Afghanistan,,33.939110,67.709953,158639,7376,0,2022-01-15 08:20:49
1,Albania,,41.153300,20.168300,228777,3262,0,2022-01-15 08:20:49
2,Algeria,,28.033900,1.659600,224979,6393,0,2022-01-15 08:20:49
3,Andorra,,42.506300,1.521800,29888,142,0,2022-01-15 08:20:49
4,Angola,,-11.202700,17.873900,93302,1852,0,2022-01-15 08:20:49
...,...,...,...,...,...,...,...,...
782,US,Ohio,40.388783,-82.764915,2307691,30922,0,2022-01-15 08:20:49
783,US,Alabama,32.318200,-86.902300,1004622,16641,0,2022-01-15 08:20:49
784,US,North Carolina,35.630066,-79.806419,2011302,19903,0,2022-01-15 08:20:49
785,US,District of Columbia,38.897438,-77.026817,121871,1255,0,2022-01-15 08:20:49


In [5]:
dataset.describe()

,Lat,Long_,Confirmed,Deaths,Recovered
count,767.000000,767.000000,7.860000e+02,786.000000,786.0
mean,26.925203,11.973154,4.118320e+05,7035.250636,0.0
std,25.443832,78.241531,1.077371e+06,16768.651731,0.0
min,-52.368000,-178.116500,0.000000e+00,0.000000,0.0
25%,9.226350,-68.381600,2.430200e+04,220.500000,0.0
50%,33.856892,14.666400,9.649300e+04,1883.000000,0.0
75%,47.175159,73.365347,3.444070e+05,6675.000000,0.0
max,71.706900,178.065000,1.327950e+07,155617.000000,0.0


In [6]:
df_total = dataset.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum"
    }
)

In [7]:
df_total

,Country_Region,Confirmed,Deaths,Recovered
0,Afghanistan,158639,7376,0
1,Albania,228777,3262,0
2,Algeria,224979,6393,0
3,Andorra,29888,142,0
4,Angola,93302,1852,0
...,...,...,...,...
191,Vietnam,1991484,35341,0
192,West Bank and Gaza,474234,5005,0
193,Yemen,10233,1988,0
194,Zambia,293695,3844,0


In [8]:
total_confirmed = dataset["Confirmed"].sum()
total_recovered = dataset["Recovered"].sum()
total_deaths = dataset["Deaths"].sum()

In [9]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

In [10]:
df_top10

,Country_Region,Confirmed,Deaths,Recovered
182,US,64964816,849259,0
23,Brazil,22933289,621063,0
79,India,36850962,485752,0
144,Russia,10565484,313458,0
115,Mexico,4257776,300912,0
138,Peru,2512789,203302,0
186,United Kingdom,15165209,152107,0
80,Indonesia,4269740,144163,0
85,Italy,8356514,140548,0
81,Iran,6217320,132026,0


In [11]:

fig1 = make_subplots( rows = 4, cols = 6,
    specs = [
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)



In [12]:
message = dataset["Country_Region"] + " " + dataset["Province_State"] + "<br>"
message += "Confirmed: " + dataset["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + dataset["Deaths"].astype(str) + "<br>"
message += "Recovered: " + dataset["Recovered"].astype(str) + "<br>"
message += "Last updated: " + dataset["Last_Update"].astype(str)
dataset["text"] = message

In [13]:
fig1.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = dataset["Long_"],
        lat = dataset["Lat"],
        hovertext = dataset["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = dataset['Confirmed'],
            cmax = dataset['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)



In [14]:
""" And for Top 10 """


fig2 = make_subplots( rows = 4, cols = 6,
    specs = [
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)


message += "Confirmed: " + df_top10["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_top10["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_top10["Recovered"].astype(str) + "<br>"
df_top10["text"] = message



In [ ]:
fig2.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        hovertext = df_top10["text"],
        showlegend=False,
        marker = dict(
            size = 10,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            color = df_top10['Confirmed'],
            cmax = df_top10['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),
    
    row=1, col=1
)

fig2.update_layout(
    xaxis=dict(
        rangeslider=dict (
            visible=True
        )
    )
)


# View of summary plots

---



In [22]:
fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

fig1.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

fig1.show()

In [ ]:
fig1.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig1.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig1.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

fig1.show()

# Full Visualization

---




In [20]:
fig1.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(dataset["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y=0.8),
    geo = dict(
            projection_type="orthographic",
            showcoastlines=True,
            landcolor="white", 
            showland= True,
            showocean = True,
            lakecolor="LightBlue"
    ),

    annotations=[
        dict(
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0.35,
            y=0)
    ]
)